# [Alpha Vantage](https://www.alphavantage.co/documentation/#news-sentiment)
Limitations:
* 25 calls/day (they seem to block based on fingerprint, not key, so try to **change runtime on Colab for each ticker**)
* max 1000 results/call

In [1]:
#arguments

av_key = 'LFZXFLHI45IN7AXV'
av_ticker = 'CRYPTO:BTC' #'CRYPTO:BTC' 'CRYPTO:ETH' 'CRYPTO:SOL'
av_from = '20120101T0000' #YYYYMMDDTHHMM
av_to = '' if av_ticker=='CRYPTO:SOL' else '20240101T0000' #Llama 3.1 knowledge cutoff
av_limit = 1000

In [2]:
#collect

import pandas as pd
import requests
from itertools import count

av_df = pd.DataFrame()

#first call
url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={av_ticker}&apikey={av_key}&time_from={av_from}&time_to={av_to}&limit={av_limit}'
av_data = requests.get(url).json()
av_feed = [{**item, 'ticker': av_ticker.split(':')[-1]} for item in av_data['feed']] #error if rate limit reached
print('\n' + av_ticker, end=': 1 ')
av_df = pd.concat([av_df, pd.DataFrame(av_feed)], ignore_index=True)

#subsequent calls
for av_call in count(2):
    av_last = av_df['time_published'].iat[-1][:-2] #trims seconds
    url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={av_ticker}&apikey={av_key}&time_from={av_from}&time_to={av_last}&limit={av_limit}'
    av_data = requests.get(url).json()
    if not 'feed' in av_data:
        print('rate limit reached', end='')
        break
    print(av_call, end=' ')
    av_feed = [{**item, 'ticker':av_ticker.split(':')[-1]} for item in av_data['feed'][1:]] #skips first one
    if not av_feed:
        break
    av_df = pd.concat([av_df, pd.DataFrame(av_feed)], ignore_index=True)


CRYPTO:BTC: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 rate limit reached

In [3]:
#show

av_df

,title,url,time_published,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment,ticker
0,Crypto Market Insights: Turning A Corner In 2024,https://www.forbes.com/sites/digital-assets/20...,20231231T233002,[Sean Lee],2023 was quite a turbulent year for the digita...,https://imageio.forbes.com/specials-images/ima...,Forbes,n/a,www.forbes.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",0.243579,Somewhat-Bullish,"[{'ticker': 'BLK', 'relevance_score': '0.05122...",BTC
1,Crypto entrepreneurs: Top winners and losers o...,https://cointelegraph.com/news/crypto-entrepre...,20231231T230000,[Robert D. Knight],Here's a look at some of the biggest winners a...,https://images.cointelegraph.com/cdn-cgi/image...,Cointelegraph,n/a,cointelegraph.com,"[{'topic': 'Blockchain', 'relevance_score': '0...",0.014941,Neutral,"[{'ticker': 'CRYPTO:BTC', 'relevance_score': '...",BTC
2,Cross-chain protocol Orbit Bridge reportedly s...,https://cointelegraph.com/news/cross-chain-pro...,20231231T223411,[Tom Mitchelhill],Hackers appear to have stolen a total of $81.7...,https://images.cointelegraph.com/cdn-cgi/image...,Cointelegraph,n/a,cointelegraph.com,"[{'topic': 'Blockchain', 'relevance_score': '0...",-0.035304,Neutral,"[{'ticker': 'CRYPTO:BTC', 'relevance_score': '...",BTC
3,Countries around the world are passing crypto ...,https://www.cnbc.com/2023/12/31/state-of-crypt...,20231231T204635,"[Ryan Browne, MacKenzie Sigalos]",In a year that saw crypto giant Binance hit wi...,https://image.cnbcfm.com/api/v1/image/10732331...,CNBC,Top News,www.cnbc.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",0.055408,Neutral,"[{'ticker': 'MA', 'relevance_score': '0.016761...",BTC
4,"Bitcoin, After 18 Months Of Knockdowns, Enters...",https://www.forbes.com/sites/brianroberts/2023...,20231231T192745,[Brian Roberts],"If cryptocurrency were a sport, it would be bo...",https://imageio.forbes.com/specials-images/ima...,Forbes,n/a,www.forbes.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",0.016926,Neutral,"[{'ticker': 'MSTR', 'relevance_score': '0.1395...",BTC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18862,Cryptocurrency Bitcoin Cash's Price Increased ...,https://www.benzinga.com/markets/cryptocurrenc...,20221205T150059,[Benzinga Insights],"Over the past 24 hours, Bitcoin Cash's BCH/USD...",,Benzinga,Trading,www.benzinga.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.074949,Neutral,"[{'ticker': 'CRYPTO:BTC', 'relevance_score': '...",BTC
18863,KuCoin Hires Auditor To Deliver Report On Exch...,https://www.benzinga.com/markets/cryptocurrenc...,20221205T150028,[Murtuza Merchant],Major cryptocurrency exchange KuCoin has hired...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Blockchain', 'relevance_score': '0...",0.130181,Neutral,"[{'ticker': 'CRYPTO:BTC', 'relevance_score': '...",BTC
18864,Bitcoin Surpasses This Key Level; Axie Infinit...,https://www.benzinga.com/markets/cryptocurrenc...,20221205T144918,[Lisa Levin],"Crypto prices traded higher this morning, with...",https://cdn.benzinga.com/files/images/story/20...,Benzinga,Trading,www.benzinga.com,"[{'topic': 'Blockchain', 'relevance_score': '0...",0.054505,Neutral,"[{'ticker': 'CRYPTO:BTC', 'relevance_score': '...",BTC
18865,Mark Cuban Joins Elon Musk In Defending Dogeco...,https://www.benzinga.com/markets/cryptocurrenc...,20221205T144622,[Shivdeep Dhaliwal],Dallas Mavericks owner Mark Cuban said on Wedn...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",0.026328,Neutral,"[{'ticker': 'TSLA', 'relevance_score': '0.1210...",BTC


In [4]:
#save

name = av_ticker.replace(':', '_').lower()+'.csv'
av_df[['ticker', 'title', 'summary']].rename(columns={'title': 'headline', 'summary': 'preview'}).to_csv(name, index=False)

# [Finnhub](https://finnhub.io/docs/api/company-news)
Limitations:
* 60 calls/minute
* max 1 year of historical data
* US companies only (no crypto etc.)

In [5]:
# pip install finnhub-python

In [9]:
#arguments

fh_key = 'cqnr07hr01qo8864qbpgcqnr07hr01qo8864qbq0'
fh_tickers = [
    'AAPL', 'MSFT', 'NVDA', #information technology
    'JPM', 'V', 'MA', #financials
    'LLY', 'JNJ', 'PFE', #health care
    'AMZN', 'TSLA', #consumer discretionary
    'META', 'NFLX', #communication services
    'GE', 'CAT', 'UBER', #industrials
    'PG', 'COST', 'WMT', 'KO', #consumer staples
    'XOM', #energy
    'NEE', #utilities
    'PLD', #real Estate
    'LIN'] #materials
fh_from = '2023-01-01' #YYYY-MM-DD
fh_to = '2025-01-01'

In [10]:
#collect

import finnhub
import pandas as pd
from itertools import count
from time import sleep

client = finnhub.Client(fh_key)
fh_df = pd.DataFrame()
for fh_ticker in fh_tickers:

    #first call
    fh_data = client.company_news(fh_ticker, fh_from, fh_to)
    print('\n' + fh_ticker, end=': 1 ')
    fh_df = pd.concat([fh_df, pd.DataFrame(fh_data)], ignore_index=True)

    #subsequent calls
    for fh_call in count(2):
        sleep(1)
        fh_last = pd.Timestamp(fh_df['datetime'].iat[-1] - 86400, unit='s').strftime('%Y-%m-%d') #day before
        fh_data = client.company_news(fh_ticker, fh_from, fh_last)
        print(fh_call, end=' ')
        if not fh_data:
            break
        fh_df = pd.concat([fh_df, pd.DataFrame(fh_data)], ignore_index=True)

fh_df['datetime'] = pd.to_datetime(fh_df['datetime'], unit='s')


AAPL: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 
MSFT: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 
NVDA: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 
JPM: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 
V: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 
MA: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 
LLY: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 
JNJ: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
PFE: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 
AMZN: 1 2 3 

In [11]:
#show

fh_df

,category,datetime,headline,id,image,related,source,summary,url
0,company,2024-09-10 13:40:01,How AI features of iPhone and rival smartphone...,129805077,,AAPL,Finnhub,Apple unveiled a raft of AI-powered features f...,https://finnhub.io/api/news?id=96b0ea901b09f33...
1,company,2024-09-10 12:59:53,EU triumphs in court fight with Google and App...,129804213,,AAPL,Finnhub,LONDON — Google lost its last bid to overturn ...,https://finnhub.io/api/news?id=799b3edde809d28...
2,company,2024-09-10 12:26:44,"EU flexes regulatory muscle, forcing Google an...",129803507,,AAPL,Finnhub,LONDON — Google lost its last bid to overturn ...,https://finnhub.io/api/news?id=165265588309247...
3,company,2024-09-10 11:24:31,Apple: Wedbush Securities raises its price target,129802585,,AAPL,Finnhub,Wedbush Securities announced on Tuesday that i...,https://finnhub.io/api/news?id=56522aa26f47bcd...
4,company,2024-09-10 11:19:05,Google and Apple have to pay billions in fines...,129802535,,AAPL,Finnhub,LONDON — Google lost its last bid to overturn ...,https://finnhub.io/api/news?id=aa28086fa547807...
...,...,...,...,...,...,...,...,...,...
144165,company,2023-09-19 03:38:00,"Linde PLC stock rises Tuesday, outperforms market",122680821,,LIN,MarketWatch,Looking for stock market analysis and research...,https://finnhub.io/api/news?id=1365718ce42ae0e...
144166,company,2023-09-18 03:38:00,"Linde PLC stock falls Monday, underperforms ma...",122664014,,LIN,MarketWatch,Looking for stock market analysis and research...,https://finnhub.io/api/news?id=328b478427593bd...
144167,company,2023-09-17 21:34:00,"SECOND DEEP DRILL HOLE INTERCEPTS 1,000 METRES...",122641909,,LIN,PR Newswire,Looking for stock market analysis and research...,https://finnhub.io/api/news?id=723a35b22ecac0c...
144168,company,2023-09-17 07:33:42,"Dividend Champion, Contender, And Challenger H...",122633800,https://static.seekingalpha.com/cdn/s3/uploads...,LIN,SeekingAlpha,Dividend Champions list is a monthly compilati...,https://finnhub.io/api/news?id=7172fb3b1d742ba...


In [12]:
#save

fh_df[['related', 'headline', 'summary']].rename(columns={'related': 'ticker', 'summary': 'preview'}).to_csv('stocks.csv', index=False)